<a href="https://colab.research.google.com/github/Jahan08/Amber-tutorial/blob/main/Retrieve-Data-Cancer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
! pip install chembl_webresource_client

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [20]:
import pandas as pd
from chembl_webresource_client.new_client import new_client

In [21]:
target = new_client.target
target_query = target.search('A498')
targets = pd.DataFrame.from_dict(target_query)
targets

,cross_references,organism,pref_name,score,species_group_flag,target_chembl_id,target_components,target_type,tax_id
0,[],Homo sapiens,A498,20.0,False,CHEMBL614516,[],CELL-LINE,9606


In [22]:
selected_target = targets.target_chembl_id[0]
selected_target

'CHEMBL614516'

In [23]:
activity = new_client.activity
res = activity.filter(target_chembl_id=selected_target).filter(standard_type="GI50")

In [ ]:
df = pd.DataFrame.from_dict(res)
df

,activity_comment,activity_id,activity_properties,assay_chembl_id,assay_description,assay_type,assay_variant_accession,assay_variant_mutation,bao_endpoint,bao_format,...,target_organism,target_pref_name,target_tax_id,text_value,toid,type,units,uo_units,upper_value,value
0,None,36426,[],CHEMBL619313,In vitro cytotoxicity against human tumor A498...,F,None,None,BAO_0000190,BAO_0000219,...,Homo sapiens,A498,9606,None,None,IC50,uM,UO_0000065,None,7.5
1,None,44807,[],CHEMBL619313,In vitro cytotoxicity against human tumor A498...,F,None,None,BAO_0000190,BAO_0000219,...,Homo sapiens,A498,9606,None,None,IC50,uM,UO_0000065,None,53.0
2,None,49899,[],CHEMBL619313,In vitro cytotoxicity against human tumor A498...,F,None,None,BAO_0000190,BAO_0000219,...,Homo sapiens,A498,9606,None,None,IC50,uM,UO_0000065,None,63.5
3,None,56138,[],CHEMBL619313,In vitro cytotoxicity against human tumor A498...,F,None,None,BAO_0000190,BAO_0000219,...,Homo sapiens,A498,9606,None,None,IC50,uM,UO_0000065,None,54.6
4,None,63979,[],CHEMBL619313,In vitro cytotoxicity against human tumor A498...,F,None,None,BAO_0000190,BAO_0000219,...,Homo sapiens,A498,9606,None,None,IC50,uM,UO_0000065,None,54.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
634,None,23219306,"[{'comments': None, 'relation': '=', 'result_f...",CHEMBL4824411,Antiproliferative activity against human A498 ...,F,None,None,BAO_0000190,BAO_0000219,...,Homo sapiens,A498,9606,None,None,IC50,uM,UO_0000065,None,25.08
635,None,23219307,"[{'comments': None, 'relation': '=', 'result_f...",CHEMBL4824411,Antiproliferative activity against human A498 ...,F,None,None,BAO_0000190,BAO_0000219,...,Homo sapiens,A498,9606,None,None,IC50,uM,UO_0000065,None,0.81
636,None,23219571,"[{'comments': None, 'relation': '=', 'result_f...",CHEMBL4824431,Antiproliferative activity against human A498 ...,F,None,None,BAO_0000190,BAO_0000219,...,Homo sapiens,A498,9606,None,None,IC50,uM,UO_0000065,None,1.0
637,None,23248284,"[{'comments': None, 'relation': '=', 'result_f...",CHEMBL4830461,Antiproliferative activity against human A498 ...,F,None,None,BAO_0000190,BAO_0000219,...,Homo sapiens,A498,9606,None,None,IC50,nM,UO_0000065,None,96.0


In [76]:
df = pd.read_csv('/content/bioactivity_data_RXF_393.csv', header = 0)

<ipython-input-76-0522ce71247c>:1: DtypeWarning: Columns (15,25,28,41) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/bioactivity_data_RXF_393.csv', header = 0)


In [77]:
# considering nM unit for MIC
df = df[(df['standard_type'] == 'GI50')]
df = df[(df['standard_type'] == 'IC50')]
#df = df[(df['standard_type'] == 'EC50')]
df = df[(df['standard_units'] == 'nM')]
df = df[(df['relation'] == '=')]
df = df[df.canonical_smiles.notna()]
df = df[df.standard_value.notna()]
df

,activity_comment,activity_id,activity_properties,assay_chembl_id,assay_description,assay_type,assay_variant_accession,assay_variant_mutation,bao_endpoint,bao_format,...,target_organism,target_pref_name,target_tax_id,text_value,toid,type,units,uo_units,upper_value,value


In [70]:
fix = []

for i in df.standard_value:
    fix.append(float(i))

STATUS = []

for i in fix:
    if i < 1000:
        STATUS.append("Yes") #active
        
    elif 30000 <= i <= 10000:
        STATUS.append("No") #inactive
        
    else:
        STATUS.append("IN") #intermediate     

df = df.drop('value', 1)
df['value'] = fix
df['Activity'] = STATUS
df

<ipython-input-70-14f59232464e>:18: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df = df.drop('value', 1)


,activity_comment,activity_id,activity_properties,assay_chembl_id,assay_description,assay_type,assay_variant_accession,assay_variant_mutation,bao_endpoint,bao_format,...,target_pref_name,target_tax_id,text_value,toid,type,units,uo_units,upper_value,value,Activity
10,NaN,138330,[],CHEMBL772327,Tested for cytotoxicity against RXF-393 cell l...,F,NaN,NaN,BAO_0000189,BAO_0000219,...,RXF 393,9606,NaN,NaN,Log GI50,NaN,UO_0000065,NaN,112.20,Yes
16,NaN,186179,[],CHEMBL772322,Growth inhibition of the RXF-393 Renal cancer ...,F,NaN,NaN,BAO_0000189,BAO_0000219,...,RXF 393,9606,NaN,NaN,Log GI50,NaN,UO_0000065,NaN,67608.30,IN
17,NaN,196832,[],CHEMBL772322,Growth inhibition of the RXF-393 Renal cancer ...,F,NaN,NaN,BAO_0000189,BAO_0000219,...,RXF 393,9606,NaN,NaN,Log GI50,NaN,UO_0000065,NaN,30199.52,IN
21,NaN,295802,[],CHEMBL772217,Cytotoxic activity against RFX-393 renal cance...,F,NaN,NaN,BAO_0000189,BAO_0000219,...,RXF 393,9606,NaN,NaN,Log GI50,NaN,UO_0000065,NaN,20417.38,IN
24,NaN,310169,[],CHEMBL772325,In vitro inhibition of tumor cell growth of RX...,F,NaN,NaN,BAO_0000189,BAO_0000219,...,RXF 393,9606,NaN,NaN,Log GI50,NaN,UO_0000065,NaN,21877.62,IN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41783,NaN,23282647,"[{'comments': None, 'relation': '=', 'result_f...",CHEMBL4837606,Growth inhibition of human RXF 393 cells measu...,F,NaN,NaN,BAO_0000189,BAO_0000219,...,RXF 393,9606,NaN,NaN,GI50,nM,UO_0000065,NaN,2310.00,IN
41784,NaN,23282648,"[{'comments': None, 'relation': '=', 'result_f...",CHEMBL4837606,Growth inhibition of human RXF 393 cells measu...,F,NaN,NaN,BAO_0000189,BAO_0000219,...,RXF 393,9606,NaN,NaN,GI50,nM,UO_0000065,NaN,5.00,Yes
41785,NaN,23282649,"[{'comments': None, 'relation': '=', 'result_f...",CHEMBL4837606,Growth inhibition of human RXF 393 cells measu...,F,NaN,NaN,BAO_0000189,BAO_0000219,...,RXF 393,9606,NaN,NaN,GI50,nM,UO_0000065,NaN,412.00,Yes
41786,NaN,23282650,"[{'comments': None, 'relation': '=', 'result_f...",CHEMBL4837606,Growth inhibition of human RXF 393 cells measu...,F,NaN,NaN,BAO_0000189,BAO_0000219,...,RXF 393,9606,NaN,NaN,GI50,nM,UO_0000065,NaN,709.00,Yes


In [71]:
# considering nM unit for MIC
df = df[(df['Activity'] != 'IN')]
len(df)

347

In [72]:

df.to_csv('/content/bioactivity_data_RXF_393_GI50_new.csv', index=False)
     